In [90]:
from langchain.document_loaders import CSVLoader
from langchain_qdrant import QdrantVectorStore
from langchain_community.vectorstores import Chroma
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from uuid import uuid4
load_dotenv()

True

In [91]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY "] = os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["QDRANT_URL"] = os.getenv("QDRANT_URL")
os.environ["QDRANT_API_KEY"] = os.getenv("QDRANT_API_KEY")




In [92]:
# import pandas as pd
# df =pd.read_csv("data/conversation_data.csv")
# df.drop("ID", axis=1, inplace=True)

# df.to_csv("data/no_id_conv.csv")

In [93]:
loader = CSVLoader(
    file_path= "data/no_id_conv.csv",
    csv_args={
        "delimiter": ",",

    },
 
)


docs=  loader.load()
docs[0]




Document(metadata={'source': 'data/no_id_conv.csv', 'row': 0}, page_content=": 0\nPatient_Answer: I so I've just had this pain in my chest for just over a week now and it's caused me to have trouble breathing.\nDoctor_response: OK, um how old are you, Jen?")

In [94]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=300)
texts = text_splitter.split_documents(docs)


In [95]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\USER\Documents\AWADOC\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# QDRANT_URL = os.getenv("QDRANT_URL")
# QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")


# qdrant = QdrantVectorStore.from_documents(
#     docs,
#     hf,
#     url=QDRANT_URL,
#     prefer_grpc=False,
#     force_recreate= True,
#     api_key=QDRANT_API_KEY,
#     collection_name="awadoc_demo",
# )

ResponseHandlingException: The write operation timed out

In [ ]:
# qdrant = QdrantVectorStore.from_existing_collection(
#     embedding=embeddings,
#     collection_name="my_documents",
#     url="http://localhost:6333",
# )

In [102]:
db = Chroma.from_documents(texts, hf,persist_directory="data/")
retriever =db.as_retriever(search_type="mmr", search_kwargs={"k":4})


In [99]:
db.persist()

C:\Users\USER\AppData\Local\Temp\ipykernel_9232\123899826.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [100]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant",
    temperature=0,)

template = """You are a professional and empathetic medical assistant designed to understand and respond to patients based on previous doctor-patient conversations.

You are given context in the form of a past conversation between a patient and a doctor. Use this context to guide your understanding and responses. 
Answer the question based on the cotext and ChatHistory. Especially take the latest question into consideration.
Your responsibilities:

1. Carefully read the provided conversation between the patient and doctor.
2. Use the dialogue to understand the patient's symptoms, history, and concerns.
3. When the patient's input is unclear or incomplete, ask follow-up questions to gain full clarity before responding.
4. Respond like a human doctor would — with clarity, compassion, and professionalism.
5. Do NOT give final diagnoses. Instead, suggest possibilities, give helpful information, and recommend next steps.
6. If the conversation lacks enough information, politely ask the patient to clarify or provide more details.
7. Do not hallucinate or make up facts. Stick to the context or ask the right questions to gather more information.
8. Keep your language friendly, clear, and medically sound.

Context (a conversation between a patient and doctor):

ChatHistory: {history}
{context}

Begin the conversation based on this context.

Patient: {question}

"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = prompt|llm


In [ ]:
from pydantic import BaseModel, Field
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain.schema import Document
from typing import List

class AgentState(BaseModel):
    messages: List(BaseMessage)
    documents: List(Document)
    on_topic: str
    rephrased_question:str
    proceed_to_generate:bool
    rephrase_count: int
    question:HumanMessage

class GradeQuestion(BaseModel):
    score: str = Field(description="Is question about the specific topic? If yes ->Yes If not -> No")

def question_rewriter(state: AgentState):
    state['documents'] = []
    state['on_topic'] = ""
    state['rephrased_question'] = ""
    state['proceed_to_generate'] = ""
    state['rephrase_count'] = 0

    if 'messages' not in state or state['messages'] is None:
        state['messages'] = []

    if state['question'] not in state['messages']:
        state['messages'].append(state['question'])

    if len(state['messages']) >1:
        conversation = state['messages'][:-1]
        current_question = state["question"].content

        messages = [
            SystemMessage(
                content= "You are a helpful assistant that rephrases the users question to be a standalone question" \
                "optimized for retrieval"
            )
        ]

        messages.extend(conversation)
        messages.append(HumanMessage(content= current_question))

        rephrase_prompt = ChatPromptTemplate.from_messages(messages)
        prompt = rephrase_prompt.format()
        response = llm.invoke(prompt)
        better_question = response.content.strip()
        print(f"question_rewriter: Rephrased question: {better_question}")
        state['rephrased_question'] = better_question

    else:
        state['rephrased_question'] = state['question'].content
    return state


def question(state: AgentState):
    print("Entering question classifier")
    system_message = SystemMessage(
        content= """
    You are a classifier that determines whether a question is medical-related.

    Instructions:
    - If the question is related to medicine, health, symptoms, diseases, treatments, drugs, anatomy, or anything in the medical domain, return: **Yes**
    - If the question is not related to medicine or health, return: **No**

    Only respond with one word: **Yes** or **No**.
    "

    """)

    human_message = HumanMessage(
        content=f"User question :{state['rephrased_quesion']}"
    )

    grade_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    structured_llm = llm.with_structured_output(GradeQuestion)
    grader_llm = grade_prompt | structured_llm
    result = grader_llm.invoke({})
    state["on_topic"] = result.score.strip()
    print(f"question_classifier: on_topic = {state['on_topic']}")
    return state


def on_topic_router(state: AgentState):
    print("Entering on topic_router")
    on_topic = state.get("on_topic", "").strip().lower()
    if on_topic =="yes":
        print("Routing to retrieve")
        return "retriever"
    else:
        print("Routing to off_topic response")
        return "off_topic_response"


    
